# Aggregate Data
This notebook combines the hourly data into daily data. The benifift of this is that the hourly data contains timing errors when merging the dataset. Keeping the data hourly would increase the number of samples, but also increase the number of bad samples.

We will also calculate the solar efficiency rather than having a actual output. This makes the data more readable and trainable for our model.

First we read in our data we collected.

In [156]:
import numpy as np
import pandas as pd

df = pd.read_csv("out.csv")

Next we convert the clearsky values (maximum output) and the actual collected output into efficiency percentages. This will make comparing days easier.

In [157]:
# make percent attained row
df["dhi_efficiency"] = df["DHI"]/df["Clearsky DHI"]
df["dni_efficiency"] = df["DNI"]/df["Clearsky DNI"]
df["ghi_efficiency"] = df["GHI"]/df["Clearsky GHI"]

## Aggregate hours into days
Next we are setting the index of our dataset to the `DATE` column. This will make it easier to group together rows by day.

In [158]:
import datetime as dt
df['DATE'] =  pd.to_datetime(df['DATE'], format='%Y-%m-%d %H:%M:%S')
df = df.set_index('DATE')

Here we are removing all hours of the day where there was no sunlight. To do this we use a little trick in our dataset. The efficiency values we created are based off the clearsky models which are directly correlated with sunlight. Thus when we preformed the division, non-sunlight hours had an efficiacy of NaN. We can exploit this by removing all columns that returned null, or the night hours.

In [159]:
grouped = df[df['dhi_efficiency'].notnull()]  
grouped = grouped.groupby('STATION')

Finally we aggregate the data into days. For most columns we take the average, except for meta data values such as long, lat, and elevation. We will deal with the categorical data (cloud and weather type) next.

In [160]:
# the interval to aggregate on
agg_intv = 'D'

daily_summary = pd.DataFrame()

daily_summary['latitude'] = grouped['latitude'].resample(agg_intv).last()
daily_summary['longitude'] = grouped['longitude'].resample(agg_intv).last()
daily_summary['elevation'] = grouped['elevation'].resample(agg_intv).last()

daily_summary['temperature'] = grouped.Temperature.resample(agg_intv).mean()
daily_summary['dew_point'] = grouped['Dew Point'].resample(agg_intv).mean()
daily_summary['relative_humidity'] = grouped['Relative Humidity'].resample(agg_intv).mean()
daily_summary['daily_precipitation'] = grouped['HourlyPrecipitation'].resample(agg_intv).mean()
daily_summary['station_pressure'] = grouped.HourlyStationPressure.resample(agg_intv).mean()
daily_summary['relative_humidity'] = grouped['Relative Humidity'].resample(agg_intv).mean()
daily_summary['wind_direction'] = grouped['Wind Direction'].resample(agg_intv).mean()
daily_summary['wind_speed'] = grouped['Wind Speed'].resample(agg_intv).mean()
daily_summary['hourly_visibility'] = grouped['HourlyVisibility'].resample(agg_intv).mean()
daily_summary['cloud_cover'] = grouped['cloud_cover'].resample(agg_intv).mean()

daily_summary['cloud_type'] = grouped['cloud_type'].resample(agg_intv).apply(pd.array)
daily_summary['weather_type'] = grouped['weather_type'].resample(agg_intv).apply(pd.array)

daily_summary['dhi_efficiency'] = grouped['dhi_efficiency'].resample(agg_intv).mean()
daily_summary['dni_efficiency'] = grouped['dni_efficiency'].resample(agg_intv).mean()
daily_summary['ghi_efficiency'] = grouped['ghi_efficiency'].resample(agg_intv).mean()

## Flatten categorical data
First we import the lookup json files that we used in the first notebook. We use these to grab an array of every possible value each cell could be. 

In [161]:
import json

cloud_types = []
with open('cloud_lookup.json') as json_file:
    cloud_json = json.load(json_file).values()
    for x in cloud_json:
        cloud_types.append(x['cloud_str'])
    cloud_types = list(dict.fromkeys(cloud_types))
    
with open('weather_lookup_converter.json') as json_file:
    weather_types = list(json.load(json_file).keys())

Now we add each of the possible cloud and weather types to the dataset as its own column. Then we add the count for each type into its respective column. Then we join this flattened data with the main dataset.

In [162]:
# seperate cloud column    
cloud_sep = pd.DataFrame([x for x in daily_summary['cloud_type'].apply(
    lambda item: dict(map(lambda x: (x, np.count_nonzero(item.to_numpy() == x)), item))).values]).fillna(0)

cloud_sep = cloud_sep.append(pd.DataFrame(columns = cloud_types)).fillna(0.0)
cloud_sep = cloud_sep.loc[:, cloud_sep.columns.notnull()]
cloud_sep = cloud_sep.set_index(daily_summary.index)
daily_summary = daily_summary.join(cloud_sep)

# seperate weather column    
weather_sep = pd.DataFrame([x for x in daily_summary['weather_type'].apply(
    lambda item: dict(map(lambda x: (x, np.count_nonzero(item.to_numpy() == x)), item))).values]).fillna(0)

weather_sep = weather_sep.append(pd.DataFrame(columns = weather_types)).fillna(0.0)
weather_sep = weather_sep.loc[:, weather_sep.columns.notnull()]
weather_sep = weather_sep.set_index(daily_summary.index)
daily_summary = daily_summary.join(weather_sep)

daily_summary.head()

latitude  longitude  elevation  temperature  dew_point  \
STATION DATE                                                                 
121     2017-01-01     32.15   -111.167      737.0     9.555556   3.555556   
        2017-01-02     32.15   -111.167      737.0    11.888889   0.444444   
        2017-01-03     32.15   -111.167      737.0    16.111111  -0.444444   
        2017-01-04     32.15   -111.167      737.0    17.888889   1.111111   
        2017-01-05     32.15   -111.167      737.0    17.444444   4.111111   

                    relative_humidity  daily_precipitation  station_pressure  \
STATION DATE                                                                   
121     2017-01-01          75.717778                  NaN         27.327778   
        2017-01-02          50.391111                  NaN         27.564444   
        2017-01-03          36.868889                  NaN         27.617778   
        2017-01-04          36.750000                  NaN         27.560000   
        2017-01-05          46.236667                  NaN         27.436667   

                    wind_direction  wind_speed  hourly_visibility  \
STATION DATE                                                        
121     2017-01-01      243.444444    4.755556               10.0   
        2017-01-02      206.288889    2.622222               10.0   
        2017-01-03      205.566667    1.566667               10.0   
        2017-01-04      243.244444    1.911111               10.0   
        2017-01-05      257.822222    4.155556               10.0   

                    cloud_cover  \
STATION DATE                      
121     2017-01-01     0.488889   
        2017-01-02     0.000000   
        2017-01-03     0.000000   
        2017-01-04     0.000000   
        2017-01-05     0.000000   

                                                           cloud_type  \
STATION DATE                                                            
121     2017-01-01  [mostly_cloudy, mostly_clear, mostly_cloudy, m...   
        2017-01-02  [clear, clear, clear, clear, clear, clear, cle...   
        2017-01-03  [clear, clear, clear, clear, clear, clear, cle...   
        2017-01-04  [clear, clear, clear, clear, clear, clear, cle...   
        2017-01-05  [clear, clear, clear, clear, clear, clear, cle...   

                                                     weather_type  \
STATION DATE                                                        
121     2017-01-01  [nan, nan, nan, nan, nan, nan, nan, nan, nan]   
        2017-01-02  [nan, nan, nan, nan, nan, nan, nan, nan, nan]   
        2017-01-03  [nan, nan, nan, nan, nan, nan, nan, nan, nan]   
        2017-01-04  [nan, nan, nan, nan, nan, nan, nan, nan, nan]   
        2017-01-05  [nan, nan, nan, nan, nan, nan, nan, nan, nan]   

                    dhi_efficiency  dni_efficiency  ghi_efficiency  \
STATION DATE                                                         
121     2017-01-01        1.447492        0.450007        0.625793   
        2017-01-02        2.387190        0.453998        0.733653   
        2017-01-03        1.693041        0.572070        0.771071   
        2017-01-04        1.157926        0.949705        0.975965   
        2017-01-05        1.003678        0.989174        0.991972   

                    mostly_cloudy  mostly_clear  clear  cloudy  partly_cloudy  \
STATION DATE                                                                    
121     2017-01-01            4.0           2.0    1.0     2.0            0.0   
        2017-01-02            0.0           0.0    9.0     0.0            0.0   
        2017-01-03            0.0           0.0    9.0     0.0            0.0   
        2017-01-04            0.0           0.0    9.0     0.0            0.0   
        2017-01-05            0.0           0.0    9.0     0.0            0.0   

                    overcast  rain_light  tstorm  drizzle  rain_heavy  rain  \
STATION DATE                                    

## Save aggregated data
To finish this notebook and save its changes, we drop unnecessary columns, clean up NaN values, and reset the index.

In [163]:
daily_summary = daily_summary.reset_index(drop=False)
daily_summary = daily_summary.drop(columns=['cloud_type', 'weather_type', 'elevation', 'wind_direction'])
daily_summary = daily_summary.dropna()
daily_summary = daily_summary.reset_index(drop=True)
daily_summary.to_parquet('solar_cleaned.parquet', engine='fastparquet', compression='GZIP')